## Submission 2 - Gradient Boosting Regression

I think the following code was used in the second submission, but I didn't take the time on making it clear and pretty since it's not in our final submission.

In [43]:
def read_files(lFilesPred, sFileLabels, sTypeLabel):
    """
    TODO 
    
    Keyword Arguments: 
    - lFilesPred: 
    - sFileLabels: 
    - sTypeLabels: 
    """
    
    # Read file labels (true labels)
    print("sFileLabels : ", sFileLabels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
    
    # Training-testing data
    iNumFiles=len(lFilesPred)
    #mPredictions=[] #np.zeros((1,iNumFiles))
    #mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 


    # We loop over the predictions files from the tsfresh and SVR
    # lDicts is a list of measurement_id and the predictions obtained 
    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
    print('sTypeLabel : ', sTypeLabel)
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
    print('dLabels')
    print(dLabels)
    
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            if dLabels[k]!='NA':
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
                for j in range(1, iNumFiles):
                    fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
                    
            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    print('mPredictions and vLabels')
    print(mPredictions.shape)
    print(len(vLabels))
    # Random forest training - regression
    
    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    return mPredictions, vLabels

In [7]:
def RandomForest_fusion(lFilesPred, sFileLabels, iEstimators, sTypeLabel, nudge, rLR, iMD,iRS):

    mPredictions, vLabels = read_files(lFilesPred, sFileLabels, sTypeLabel)
    
    clf=GradientBoostingRegressor(n_estimators=iEstimators, learning_rate=rLR, max_depth=iMD,\
                                      random_state=iRS, loss='ls').fit(mPredictions, vLabels)
    vNewPred=clf.predict(mPredictions)
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]
    
    # we will include the testing data here
    return (vNewPred, vParID, vLabels, vRes1, vRes2, clf)
#print(mPredictions)

In [8]:
def randomForest_predTest(lTestPred, clf, sDirOut,sName):
    
    lDicts=[] #Will contain the two test predictions
    lmesID=[]
    iNumFiles=len(lTestPred)
    vPredIter=np.zeros((1,iNumFiles))

    for sFilePred in lTestPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            lmesID.append(k)
            #vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
            if fPred:
                vPredIter[0,j]=float(np.asarray(fPred))
            else:
                print(['Unkwnown key:' + k])
                vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    
    vTestPred=clf.predict(mPredictions)
    df = pd.DataFrame({'measurement_id': lmesID, sName:vTestPred})
    df.to_csv(sDirOut+'submissionCisPD'+sName+'.csv', index=False)
    return(lmesID, vTestPred)



### Dyskinesia

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_on_off.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sFilePred2='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTypeLabel='on_off'
n_estimators=300
rLR=0.1
iMD=1
iRS=15
nudge=100
sFileLabels=sFilePred1

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1,\
vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel, nudge, rLR, iMD,iRS)

print('Overall MSE Classif. 1 : ', get_final_score(vTrueLabels,np.array(vParID).astype(int), vRes1))
print('Overall MSE Classif. 2 : ', get_final_score(vTrueLabels, vParID, vRes2))
print('Overall MSE Fusion : ', get_final_score(vPredCross, vParIDCross,vTrueCross))




sTestPred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/On_off_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/'
sName='on_off'
#lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
#print(vTestPred)




### Tremor - Submission 2

In [ ]:
# tsfresh predictions file 
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_tremor.csv'

# SVR predictions file 
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold_all/objs_450_kernel_linear_c_0.02_eps_0.1.csv'

# subchallenge 
sTypeLabel='tremor'

# RandomForest hyperparameter 
n_estimators=5

# Labels
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'


lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion(lFilesPred,
                                                                           sFileLabels,
                                                                           n_estimators,
                                                                           sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.tremor.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/Tremor_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='tremor'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



### Dyskinesia - Submission 2

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_dyskinesia.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sTypeLabel='dyskinesia'
n_estimators=5
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold/Dyskinesia_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='z_dyskinesia'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



# Other drafts

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_everyone_svr_bpd import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_knn_bpd2 import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
uniq_subjectid = np.unique(vTraiSubjectId)
enc = OneHotEncoder(handle_unknown='ignore').fit(uniq_subjectid.reshape(-1,1))
print(uniq_subjectid.reshape(-1,1).shape)
print(enc)
temp = enc.transform([[1004]]).toarray()
print(temp.shape)
temp2 = np.repeat(temp,61,axis=0)
temp2.shape
#temp3 = np.random.rand(61,50)
#temp4 = np.concatenate((temp3,temp2),axis=1)
#print(temp4.shape)

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_knn_bpd2 import *

# Get Predictions for Per Patient SVR

## Tremor Best Config 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/resiVecPerPatientSVR_Fold"+fold

    iNeighbors=None

    for iComponent in [250]:#[50, 100, 150, 200, 250, 350, 450, 500, 550]:

        for fCValue in [20.0]:#[2e-13, 2e-07, 0.002, 0.2, 20.0]:
            vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponent)

            pca_knn_bpd2(sFileTrai, sFileTest, sOut,
                         iComponent,
                         iNeighbors,
                         sKernel='linear',
                         fCValue=fCValue,
                         fEpsilon='0.1',
                         bLabelNormalization=False,
                         bPatientPredictionsPkl=True)


    

## Dysk Best Config 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/resiVecPerPatientSVR_Fold"+fold

    iNeighbors=None

    for iComponent in [100]:#[50, 100, 150, 200, 250, 350, 450, 500, 550]:

        for fCValue in [20.0]:#[2e-13, 2e-07, 0.002, 0.2, 20.0]:
            vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponent)

            pca_knn_bpd2(sFileTrai, sFileTest, sOut,
                         iComponent,
                         iNeighbors,
                         sKernel='linear',
                         fCValue=fCValue,
                         fEpsilon='0.1',
                         bLabelNormalization=False,
                         bPatientPredictionsPkl=True)


    

### Test with Mean Normalization SVR 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"


    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecSVR_Fold"+fold

    iNeighbors=None

    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    iComponents=500
    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=True)


    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=True)

### Per Patient - With Mean Normalization

In [ ]:
%load_ext autoreload
%autoreload 2

for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"


    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecSVR_Fold"+fold

    iNeighbors=None

    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    iComponents=500
    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=True)


    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=True)
    
# sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/"
# #sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
# bKnn=False
# bSVR=True
# bEveryoneSVR=False
# bPerSubject=False 
# sDatabase=None
# sSubchallenge=None


In [ ]:
from local.get_final_scores_accuracy import get_final_scores_SVR 
from local.get_final_scores_accuracy import get_final_scores_SVR_lowest_mse_for_subjectid

sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=True 
sDatabase='CIS'
sSubchallenge='dysk'

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

# get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

### Test WITHOUT Mean Normalization

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/resiVecSVR_Fold"+fold

    iNeighbors=None

    iComponents=500
    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=False)

    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=False)
    


In [ ]:
sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=True 
sDatabase='CIS'
sSubchallenge='dysk'

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)


#get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

In [ ]:
%load_ext autoreload
%autoreload 2

from local.get_final_scores_accuracy import get_final_scores_SVR 

sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=False 
sDatabase=None
sSubchallenge=None

get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

### SVR testing around

In [ ]:


sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold0/ivector.scp"
iComponents=400

sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/ResiVecSVR_Fold0"

iNeighbors=None

vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')


In [ ]:

%load_ext autoreload
%autoreload 2
from pca_knn_bpd import *

sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Testing_Fold0/ivector.scp" 

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/ivectors_Testing_Fold0/ivector.scp" 


iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)
print(vTraiPCA.shape)

# /home/mpgill/BeatPD/BeatPD-CLSP-JHU/pca_everyone_svr_bpd.py 

# sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
# sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"


sOut="/home/mpgill/test/"
sKernel='linear' 
fCValue=0.00000000200000 
fEpsilon=0.1
iNeighbors=None

pca_knn_bpd(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')

In [ ]:
%load_ext autoreload
%autoreload 2

from get_final_scores_accuracy import get_final_scores_SVR 

sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False

get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR)

In [ ]:
%load_ext autoreload
%autoreload 2

from get_final_scores_accuracy import get_final_scores_SVR_lowest_mse_for_subjectid 

#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"
sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False

#get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR)

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR)

In [ ]:
salut = ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_10040',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_10040',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_10040',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_10040',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_10041',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_10041',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_10041',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_10041',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_10041']

salut2= ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_hbv0401',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_hbv0401',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_hbv0401',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_hbv0401',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_hbv0411',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_hbv0411',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_hbv0411',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_hbv0412',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_hbv0412']


sPattern=r'(?<=[_])(1[0-9]{3}(?<=\d)(?!=$)|hbv[0-9]{3})'

vTraiSubjectId = [re.findall(sPattern, fileName)[0] for fileName in salut]

# vTraiSubjectId = np.array(([x[-5:-1] for x in np.array(salut)]))
print(vTraiSubjectId)

vTraiSubjectId2 = [re.findall(sPattern, fileName)[0] for fileName in salut2]
print(vTraiSubjectId2)

In [ ]:
# import pandas as pd 
# import re


# def read_log_results_to_excel(folders, fileName):
#     for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
#         print(folder)
#         value = []
#         liVecDim = [350,400]#[350,400,450,500,550, 600, 650, 700]
#         for ivecDim in liVecDim:
#             sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'

#             #config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
# #             config_pattern = re.compile(r"GLOBAL WINNER PARAMETERS(.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)")


#             textfile = open(sFilePath+fileName)
#             filetext = textfile.read()
#             textfile.close()
# #             print(filetext)
#             matches = re.findall(r"GLOBAL WINNER PARAMETERS(?:.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)",
#                                  filetext)
# #             matches = re.findall(r"GLOBAL WINNER PARAMETERS(.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)",
# #                                  filetext)
#             print(matches)
# #             with open(sFilePath+fileName) as f:
# #                 print(f)
# #                 match = config_pattern.search(line)
# #                 if match:
# #                     #print(match.groups()[0])
# #                     value.append(match.groups()[0])
# #                     #yield line.strip()

#         value = pd.DataFrame(value)
#         value = value.T
#         value.columns = liVecDim
#         display(value)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuPerPatientSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

# # folders=['on_off_noinact_auto30_320fl']
# # fileName='globalAccuEveryoneSVR_Test.log'
# # read_log_results_to_excel(folders, fileName)

In [ ]:
# import pandas as pd 
# import re


# def read_log_results_to_excel(folders, fileName):
#     for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
#         print(folder)
#         value = []
#         liVecDim = [350,400,450,500,550, 600, 650, 700]
#         for ivecDim in liVecDim:
#             sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'

#             #config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
#             config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
# #             config_pattern = re.compile(r"GLOBAL WINNER PARAMETERS(?:.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)")
#             with open(sFilePath+fileName) as f:
#                 for line in f:
#                     match = config_pattern.search(line)
#                     if match:
#                         #print(match.groups()[0])
#                         value.append(match.groups()[0])
#                         #yield line.strip()

#         value = pd.DataFrame(value)
#         value = value.T
#         value.columns = liVecDim
#         display(value)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuPerPatientSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuEveryoneSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

In [ ]:
import pandas as pd 
import re


def read_log_results_to_excel(folders, fileName):
    for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
        print(folder)
        value = []
        liVecDim = [350,450,500,550]
        for ivecDim in liVecDim:
            sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'
            print('Opening : ', sFilePath+fileName)
            textfile = open(sFilePath+fileName)
            filetext = textfile.read()
            textfile.close()
            
            
            result = re.findall(r"Test Final score\s[:| ]\s*(\d*.\d*)",filetext)
            print(result[len(result)-1])
            value.append(result[len(result)-1])
            
        value = pd.DataFrame(value)
        value = value.T
        value.columns = liVecDim
        display(value)

folders=['on_off_noinact_auto30']
fileName='globalAccuPerPatientSVR_Test.log'
read_log_results_to_excel(folders, fileName)

folders=['on_off_noinact_auto30_320fl']
fileName='globalAccuEveryoneSVR_Test.log'
read_log_results_to_excel(folders, fileName)

# Remove inactivity with max 

In [ ]:
def remove_inactivity_max(df_train_label):
    last_filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    display(last_filtered_value)
    for idx in df_train_label.index:
        df_allo = []
        df_train_data = pd.read_csv(
            path_train_data + df_train_label["measurement_id"][idx] + ".csv"
        )

        # Get the absolute max value for X, Y, Z
        max_values = df_train_data.iloc[:, -3:].abs().max()

        # Compute what is 5% of that max
        thresold_energy = 5
        df_treshold = (max_values * thresold_energy) / 100

        # display(df_train_data)
        # Candidates are the frames where X, Y, Z are below that treshold (5% of the max)
        #         df_candidates = df_train_data[(df_train_data.X.abs() <= df_treshold.X) &
        #                                      (df_train_data.Y.abs() <= df_treshold.Y) &
        #                                      (df_train_data.Z.abs() <= df_treshold.Z)]
        # display(df_candidates)
        for idx2 in df_train_data.index:
            # print('df_train_data[idx2]')
            # display(df_train_data.iloc[idx2,-3:])
            last_filtered_value = filtered_value
            filtered_value = last_filtered_value + 0.004 * (
                df_train_data.iloc[idx2, -3:] - last_filtered_value
            )
            y = pd.DataFrame(columns=["Timestamp"])
            y = pd.concat(
                [y, pd.DataFrame([df_train_data.iloc[idx2, 0]], columns=["Timestamp"])],
                ignore_index=True,
            )
            #             print('display y :')
            #             display(y)
            #             print('end display y')

            #             print('display filtered value')
            #             display(pd.DataFrame(filtered_value).transpose())
            #             print('end display filtered value')
            df_allo.append(
                pd.concat([y, pd.DataFrame(filtered_value).transpose()], axis=1)
            )
        #             print('display df_allo')
        #             display(df_allo)

        # FIXME : change the name df_allo
        df_allo = pd.DataFrame(df_allo, columns=("Timestamp", "X", "Y", "Z"))

        df_allo.plot(x="Timestamp", legend=True, subplots=True, title="allo")
        stop()


#         v_candidate_x = pd.DataFrame({'Candidate':list(0)})
#         v_candidate_x = np.where(df_train_data.X.abs() <= df_treshold.X, 1, 0)

In [7]:
from scipy.spatial.transform import Rotation as R
import numpy as np 
rot_ang = 45

rot = np.random.randint(-rot_ang,rot_ang,size=1)[0]
print('rot : ', rot)
print([rot]*3)
r = R.from_euler('xyz', [rot]*3, degrees=True)
print(r)
rot_mat = r.as_dcm()
print('rot_mat : ', rot_mat)
# temp_train_X = np.dot(temp_train_X, rot_mat)

rot :  -7
[-7, -7, -7]
rot_mat :  [[ 0.98514786  0.13570238 -0.10520719]
 [-0.12096095  0.98333784  0.13570238]
 [ 0.12186934 -0.12096095  0.98514786]]


# Tsfresh experiments

In [105]:
import pandas as pd
all_labels = pd.read_csv('/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv')

In [106]:
all_obj = ["on_off", "tremor", "dyskinesia"]
obj = "tremor"

all_labels = all_labels.drop(list(set(all_obj) - set([obj])), axis=1)

In [107]:
all_features = pd.read_csv('/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/features/cis-pd.training.csv')
all_features_labels = pd.merge(all_features, all_labels, on=["measurement_id"])

In [108]:
all_features_labels = all_features_labels.dropna(subset=[obj])

In [109]:
# Compute the average for a speaker of all the features and add those new features to all_features_label
avg = all_features_labels.groupby('subject_id').mean().reset_index().add_prefix('sp_').rename(columns={'sp_subject_id':'subject_id'})
all_features_labels = pd.merge(all_features_labels, avg, on='subject_id')


In [110]:
# Performing global normalization, removing the mean for the features and the labels
remove = []
for i in all_features_labels.columns:
    # if it doesn't start with sp and its sp_ is in  all_features_labels
    if not i.startswith('sp_') and 'sp_' + i in all_features_labels.columns:
        # We remove the sp_ column
        remove.append('sp_' + i)
        # We replace the feature with their value minus the mean of the subject 
        all_features_labels[i] = all_features_labels[i] - all_features_labels['sp_' + i]
# We drop the sp_ columns which were only useful to substract the mean
all_features_labels = all_features_labels.drop(remove, axis=1)

In [111]:
all_features_labels = pd.merge(all_features_labels, pd.read_csv('/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/data/order.csv'), how='inner', on=["measurement_id"])

In [112]:
weight = all_features_labels.groupby(['subject_id', 'fold_id']).count().reset_index()[["subject_id", "fold_id", obj]].rename(columns={obj: 'spcount'})

In [113]:
type(all_features_labels)

pandas.core.frame.DataFrame

In [114]:
all_features_labels = pd.merge(all_features_labels, weight, on=['subject_id', 'fold_id'])

In [115]:
subject_id = pd.get_dummies(all_features_labels.subject_id, columns='subject_id', prefix='spk_')

In [116]:
all_features_labels = pd.concat([all_features_labels, subject_id], axis=1)

In [117]:
all_features_labels.isnull().sum()

X2_abs_energy             0
X2_abs_sum_of_changes     0
X2_agg_autocorr_mean      0
X2_agg_autocorr_median    0
X2_agg_autocorr_var       0
                         ..
spk__1038                 0
spk__1043                 0
spk__1046                 0
spk__1048                 0
spk__1049                 0
Length: 1680, dtype: int64

In [118]:
import numpy as np 
Y = np.array(all_features_labels[obj])

In [23]:
#all_features_labels["tremor"]

In [24]:
W = np.array(all_features_labels['spcount']) ** -0.5

In [25]:
#all_features_labels['spcount']

In [26]:
#all_features_labels.keys().to_list()

In [27]:
foldid = np.array(all_features_labels['fold_id']).astype(int)
from sklearn.model_selection import PredefinedSplit
cv = PredefinedSplit(foldid)

# X contains all the features 
X = all_features_labels.drop([obj, 'subject_id', 'measurement_id', 'spcount', 'fold_id'], axis=1).astype(pd.np.float32).values
print(X.shape)
print(foldid.shape)

(1462, 1675)
(1462,)


/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [178]:
avg.keys()

Index(['subject_id', 'sp_X2_abs_energy', 'sp_X2_abs_sum_of_changes',
       'sp_X2_agg_autocorr_mean', 'sp_X2_agg_autocorr_median',
       'sp_X2_agg_autocorr_var', 'sp_X2_augmented_dickey_fuller_pvalue',
       'sp_X2_augmented_dickey_fuller_teststat',
       'sp_X2_augmented_dickey_fuller_usedlag', 'sp_X2_autocorrelation_1',
       ...
       'sp_Z_time_rev_asym_stat_1', 'sp_Z_time_rev_asym_stat_10',
       'sp_Z_time_rev_asym_stat_100', 'sp_Z_time_rev_asym_stat_1000',
       'sp_Z_time_rev_asym_stat_2', 'sp_Z_time_rev_asym_stat_3',
       'sp_Z_time_rev_asym_stat_4', 'sp_Z_var', 'sp_Z_var_larger_than_std_dev',
       'sp_tremor'],
      dtype='object', length=1664)

In [28]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
# xgboost
clf = xgb.XGBRegressor()

best_params = {'subsample': 1.0, 'silent': False, 'gamma': 1.0, 'reg_lambda': 100.0, 'min_child_weight': 0.5, 'objective': 'reg:squarederror', 'learning_rate': 0.3, 'max_depth': 2, 'colsample_bytree': 0.8, 'n_estimators': 100, 'colsample_bylevel': 0.5}



/home/mpgill/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/mpgill/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/mpgill/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:

results = []
baselines = []

preds = []

lambda_value = None
random_forest = None
all_spks = all_features_labels['subject_id'].unique()

columns_to_exclude = (all_features_labels.columns[(all_features_labels.dtypes.values != np.dtype('float64'))]).to_list()
columns_to_exclude.remove('fold_id')

for i in range(5):
    ##test = pd.read_csv(sys.argv[i]).squeeze()
    ##idx = all_features_labels['measurement_id'].isin(test)

    # Filter all_features_labels to only the measurements of the current fold
    idx = all_features_labels['fold_id'] == i

    ##tr_w = all_features_labels[~idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##tr = pd.merge(all_features_labels[~idx], tr_w, on='subject_id')
    
    # Drop the measurements that are not from the current fold 
    tr = all_features_labels[~idx].drop(['fold_id'], axis=1)

    # Data augmentation with a lambda
    if lambda_value is not None:
        for spk in all_spks: 
            # Filter training data for that speaker 
            tr_subject = tr.loc[tr['subject_id'] == spk]

            # FIXME: The mean number of recordings changes per fold so to make it easier 
            # I'm just gonna use the mean for the data augmented values
            mean_spcount = int(tr_subject['spcount'].mean())

            # Apply the lambda on the training features except the columns that are not float
            modDfObj1 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
            modDfObj2 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
            
            df_data_aug = []
            # First loop to go over the rows
            for index, measurement1 in modDfObj1.iterrows():
                # Second rows to go over the loop except the same two rows
                for index2, measurement2 in modDfObj2.iterrows():
                    if index >= index2:
                        continue
                    df_data_aug.append(measurement1.add(measurement2).to_list())
            df_data_aug = pd.DataFrame(df_data_aug, columns=modDfObj1.columns)
            
            # Add again the columns we just removed 
            #modDfObj1 = pd.concat([modDfObj1, tr_subject[columns_to_exclude]], axis=1)
            subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]
            # Select the first row of spk_ elements 
            subject_row = tr_subject.iloc[0,tr_subject.columns.str.startswith("spk_")]
            subject_row = subject_row.append(pd.Series(mean_spcount, index=["spcount"]))
            # Duplicate that first row for the number of elements we need
            # full_subjects = pd.concat([subject_row]*len(df_data_aug), ignore_index=True)
            #FIXME NEEDED?
            # full_subjects = pd.DataFrame(full_subjects, columns=subjects_columns_title)

            full_subjects = pd.concat([pd.DataFrame(subject_row).T]*len(df_data_aug), ignore_index=True)
            df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)
            #df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)

            # Append the augmented dataframe to the original dataframe
            tr = pd.concat([tr, df_data_aug], ignore_index=True)
            print('After spk ', str(spk), ' tr shape is : ', tr.shape)
#         tr.to_csv("mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value), index=False)
    # else:
        # print(tr.keys())
        # If we are using lambda data augmentation, these columns were already removed 
        # tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)
    print('obj is : ', obj)
    train_y = tr[obj].astype(pd.np.float32) # training labels 
    train_weight = tr['spcount'] ** -0.5 # training weight
    tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)


    ##te_w = all_features_labels[idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##te = pd.merge(all_features_labels[idx], te_w, on='subject_id')
    
    # Drop the measurements that are used in the training of this fold, so we keep [idx] instead of [~idx]
    te = all_features_labels[idx].drop(['fold_id'], axis=1)
    # print('te !!! : ', te)

#     print('te[spcount] : ', te['spcount'])
    test_weight = te['spcount'] ** -0.5 # test weight 
#     print('test_weigth : ', test_weight)
    test_y = te[obj].astype(pd.np.float32) # testing labels
    sub = te['subject_id']
    sid = te.subject_id
    test_measurement_id = te.measurement_id
    te = te.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    if random_forest:
        # Random Forest Regressor 
        print('Using Random Forest Regressor')
        clf = RandomForestRegressor(**best_params)
        clf.fit(tr, train_y, sample_weight=train_weight)
    else:
        # XGBoost 
        print('Using XGBOOST')
        clf = xgb.XGBRegressor(**best_params)
        #clf = xgb.XGBClassifier(**params)

        # Fit for the xgboost 
        clf.fit(
            tr, train_y,
            sample_weight=train_weight,
            eval_set=[(tr, train_y), (te, test_y)],
            #eval_metric=',
            sample_weight_eval_set=[train_weight, test_weight],
            verbose=0,
            early_stopping_rounds=100
        )
#     print(' AVG !!! ')
#     print(avg.keys())
#     print(avg.sp_tremor)
#     print('end avg')
    pred = clf.predict(te).clip(0, 4)
    # print('pred : ', pred)
    mse = (pred - test_y) ** 2
    #mse = test_y.to_numpy() ** 2
    mse2 = test_y ** 2
#     display(te)
    ret = pd.concat([sub, mse], axis=1)
#     display(ret)
    #ret.to_csv('tmp.csv')
    # Mean MSE Per subject ID 
#     print('ret.groupby(subject_id) : ', ret.groupby('subject_id'))
#     display(ret.groupby('subject_id').sum()[obj])
#     print(ret)
    mse_marie = (ret.groupby('subject_id').sum())[obj].to_numpy()
    
    # Number of files per subject_id
    cnt_marie = (ret.groupby('subject_id').count())[obj].to_numpy()
    print('mse_marie : ', mse_marie)
    print('cnt_marie : ', cnt_marie)
#     cnt = cnt ** 0.5
    msek_per_fold = (1/cnt_marie) * mse_marie
    print('msek_per_fold : ', msek_per_fold)
    
    res = pd.DataFrame(data={'measurement_id': test_measurement_id, 'subject_id': sid, obj: pred})

    # We merge the preds with the avg on subject_id, the goal is to keep as label the standardized one
    res = pd.merge(res, avg, on='subject_id')
    
    # Add the sp_obj value to tremor 
    res[obj] += res['sp_' + obj]
    # Res now only contains the measurement_id and the standardized label
    res = res[["measurement_id", obj]]

    preds.append(res)

    print('mse : ', mse)
#     print('test_weigth : ', test_weight)
#     print('before squeeze : ', ((mse * test_weight).sum() / test_weight.sum()))
#     print('after squeeze : ', ((mse * test_weight).sum() / test_weight.sum()).squeeze())
    results.append(((mse * test_weight).sum() / test_weight.sum()).squeeze())
    baselines.append(((mse2 * test_weight).sum() / test_weight.sum()).squeeze())
preds = pd.concat(preds)
print('preds ', preds)
    
print('results : ', results)
#     raise KeyboardInterrupt

obj is :  tremor
Using XGBOOST


/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


mse_marie :  [31.609755   4.3519177  8.471712   0.7555514  4.107503   9.904063
  7.962609   2.3895192  9.874815  14.578803   3.7171767  3.7183816
 12.909976 ]
cnt_marie :  [19  9 62 10 41 24 38 10 43  9 14 20 18]
msek_per_fold :  [1.66367129 0.48354642 0.13664052 0.07555514 0.100183   0.4126693
 0.20954234 0.23895192 0.22964686 1.61986701 0.26551262 0.18591908
 0.71722089]
mse :  115     0.107426
116     0.107426
117     0.107426
118     0.107426
119     0.107426
          ...   
1411    4.000000
1412    3.975427
1413    1.000000
1414    0.000038
1415    0.840090
Name: tremor, Length: 317, dtype: float32
obj is :  tremor
Using XGBOOST
mse_marie :  [18.749796   4.091872  20.029758   4.2750916  3.6546662 10.424716
  9.638859   2.1075165 11.048466  11.680545   3.748107   8.803325
 12.527713 ]
cnt_marie :  [17  8 61  9 40 22 37  8 42  8 14 20 18]
msek_per_fold :  [1.10292917 0.51148403 0.3283567  0.47501018 0.09136665 0.47385073
 0.2605097  0.26343957 0.26305871 1.46006811 0.26772193 0.440

In [129]:
# print(preds)
df_preds = pd.concat(preds)
display(df_preds)

,measurement_id,tremor
0,41440dba-2717-4c5e-b383-463888bd4a43,0.327759
1,4343b50e-e012-4357-a8da-7e5638a530f7,0.327759
2,43b147ad-6eef-4d2d-83ea-bca3b1cd0a77,0.327759
3,4412adc6-00f7-41dd-9470-d53d56f8d504,0.327759
4,472201aa-effc-42a8-8f2b-11024eb5e913,0.327759
...,...,...
261,184ffe2e-f6c6-414d-ae43-d01bcd5568d9,1.377504
262,1d624788-41fd-4c7d-b2c3-2fed920457f6,1.226730
263,1e051b85-eca9-4831-b1e3-444a4cb18269,1.246617
264,32184eb0-2e4c-49a2-8d63-c1a67f9e17a3,1.000000


In [153]:
all_labels.keys()

Index(['measurement_id', 'subject_id', 'tremor'], dtype='object')

In [157]:
true_labels = all_labels[["measurement_id", obj, 'subject_id']]
true_labels = true_labels.dropna(subset=[obj])
true_labels = true_labels.rename(columns={obj: 'true_'+obj})
preds_true_labels = pd.merge(df_preds, true_labels, on='measurement_id')
# print(all_labels[['subject_id']])
# preds_true_labels = pd.merge(preds_true_labels, all_labels[['subject_id']], on='measurement_id')
display(preds_true_labels)

,measurement_id,tremor,true_tremor,subject_id
0,41440dba-2717-4c5e-b383-463888bd4a43,0.327759,0.0,1007
1,4343b50e-e012-4357-a8da-7e5638a530f7,0.327759,0.0,1007
2,43b147ad-6eef-4d2d-83ea-bca3b1cd0a77,0.327759,0.0,1007
3,4412adc6-00f7-41dd-9470-d53d56f8d504,0.327759,0.0,1007
4,472201aa-effc-42a8-8f2b-11024eb5e913,0.327759,0.0,1007
...,...,...,...,...
1457,184ffe2e-f6c6-414d-ae43-d01bcd5568d9,1.377504,0.0,1004
1458,1d624788-41fd-4c7d-b2c3-2fed920457f6,1.226730,0.0,1004
1459,1e051b85-eca9-4831-b1e3-444a4cb18269,1.246617,0.0,1004
1460,32184eb0-2e4c-49a2-8d63-c1a67f9e17a3,1.000000,0.0,1004


In [176]:
all_spks = preds_true_labels['subject_id'].unique()
print(all_spks)
msek_subject = []
for spk in all_spks: 
    spk_preds_true_label = preds_true_labels[preds_true_labels['subject_id'] == spk]
    msek = ((spk_preds_true_label['true_'+obj] - spk_preds_true_label[obj]) ** 2).sum() * (1/len(spk_preds_true_label))
    print(spk , ' : ', msek)
    msek_subject.append(msek)

print('msek mean')
print(sum(msek_subject) / 13)
display((pd.DataFrame(msek_subject)))

[1007 1019 1020 1048 1049 1038 1032 1006 1023 1034 1046 1043 1004]
1007  :  0.2771627441227683
1019  :  0.43342443102666095
1020  :  0.19609122499895099
1048  :  0.3402718255348038
1049  :  0.6539842594602461
1038  :  0.24700469091741348
1032  :  0.26612276957660885
1006  :  0.5007043429058206
1023  :  0.3425083620903241
1034  :  0.29783571444943935
1046  :  0.24047812576314187
1043  :  1.2703156103340894
1004  :  1.4368984495448818
msek mean
0.5002155808250115


,0
0,0.277163
1,0.433424
2,0.196091
3,0.340272
4,0.653984
5,0.247005
6,0.266123
7,0.500704
8,0.342508
9,0.297836


In [168]:
msek_subject

6.502802550725149

In [158]:
(true_labels.groupby('subject_id').count())

,measurement_id,true_tremor
subject_id,,
1004,82,82
1006,37,37
1007,299,299
1019,45,45
1020,195,195
1023,106,106
1032,177,177
1034,40,40
1038,207,207


# Old experiments 

In [254]:
clf = xgb.XGBRegressor(**best_params)

In [255]:
print(tr.isnull().sum())
clf.fit(
        tr, train_y,
        sample_weight=train_weight,
        eval_set=[(tr, train_y), (te, test_y)],
        #eval_metric=',
        sample_weight_eval_set=[train_weight, test_weight],
        verbose=0,
        early_stopping_rounds=100
    )

X2_abs_energy                 0
X2_abs_sum_of_changes         0
X2_agg_autocorr_mean          0
X2_agg_autocorr_median        0
X2_agg_autocorr_var           0
                          ...  
spk__1038                 75565
spk__1043                 75565
spk__1046                 75565
spk__1048                 75565
spk__1049                 75565
Length: 1675, dtype: int64


XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=0.5,
       colsample_bynode=1, colsample_bytree=0.8, gamma=1.0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.3, max_delta_step=0, max_depth=2,
       min_child_weight=0.5, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=100.0, scale_pos_weight=1, silent=False, subsample=1.0,
       tree_method=None, validate_parameters=False, verbosity=None)

In [319]:
print(tr.describe())
print(test_y.describe())
tr.isnull().sum()

       X2_abs_energy  X2_abs_sum_of_changes  X2_agg_autocorr_mean  \
count   68945.000000           68945.000000          68945.000000   
mean        5.104468             -23.276720             -0.001439   
std       872.401739             773.552999              0.074970   
min      -857.646369           -1631.803133             -0.220339   
25%      -252.430252            -567.171440             -0.054163   
50%      -171.420277            -185.894422             -0.002371   
75%        21.401852             407.083443              0.047865   
max     17230.849708            5467.783781              0.264528   

       X2_agg_autocorr_median  X2_agg_autocorr_var  \
count            68945.000000         68945.000000   
mean                -0.002631             0.000434   
std                  0.086186             0.006365   
min                 -0.221219            -0.013496   
25%                 -0.060548            -0.003159   
50%                 -0.001015            -0.000777   


X2_abs_energy                 0
X2_abs_sum_of_changes         0
X2_agg_autocorr_mean          0
X2_agg_autocorr_median        0
X2_agg_autocorr_var           0
                          ...  
spk__1038                 67800
spk__1043                 67800
spk__1046                 67800
spk__1048                 67800
spk__1049                 67800
Length: 1679, dtype: int64

In [259]:
pred = clf.predict(te).clip(0, 4)
pred

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [395]:
lambda_value = 1

all_spks = all_features_labels['subject_id'].unique()

columns_to_exclude = (all_features_labels.columns[(all_features_labels.dtypes.values != np.dtype('float64'))]).to_list()
columns_to_exclude.remove('fold_id')

for i in range(5):
    ##test = pd.read_csv(sys.argv[i]).squeeze()
    ##idx = all_features_labels['measurement_id'].isin(test)

    # Filter all_features_labels to only the measurements of the current fold
    idx = all_features_labels['fold_id'] == i

    ##tr_w = all_features_labels[~idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##tr = pd.merge(all_features_labels[~idx], tr_w, on='subject_id')
    
    # Drop the measurements that are not from the current fold
    tr = all_features_labels[~idx].drop(['fold_id'], axis=1)
    
        # Data augmentation with a lambda
    if 5 == 6:
        print('Loading model : ', "mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value))
        if sys.argv[5] == "load_mdl":
            tr = pd.read_csv("mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value), index_col=[0])
    elif lambda_value is not None:
        for spk in all_spks: 
            # Filter training data for that speaker 
            tr_subject = tr.loc[tr['subject_id'] == spk]
            
            # FIXME: The mean number of recordings changes per fold so to make it easier 
            # I'm just gonna use the mean for the data augmented values
            mean_spcount = int(tr_subject['spcount'].mean())
            
            
            # Apply the lambda on the training features except the columns that are not float
            modDfObj1 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
            modDfObj2 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
            
            df_data_aug = []
            # First loop to go over the rows
            for index, measurement1 in modDfObj1.iterrows():
                # Second rows to go over the loop except the same two rows
                for index2, measurement2 in modDfObj2.iterrows():
                    if index >= index2:
                        continue
                    df_data_aug.append(measurement1.add(measurement2).to_list())
            df_data_aug = pd.DataFrame(df_data_aug, columns=modDfObj1.columns)
            
            # Add again the columns we just removed 
            #modDfObj1 = pd.concat([modDfObj1, tr_subject[columns_to_exclude]], axis=1)
            subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]
            # Select the first row of spk_ elements 
            subject_row = tr_subject.iloc[0,tr_subject.columns.str.startswith("spk_")]
            subject_row = subject_row.append(pd.Series(mean_spcount, index=["spcount"]))
            
            # Duplicate that first row for the number of elements we need
            # full_subjects = pd.concat([subject_row]*len(df_data_aug), ignore_index=True)
            #FIXME NEEDED?
            # full_subjects = pd.DataFrame(full_subjects, columns=subjects_columns_title)

            full_subjects = pd.concat([pd.DataFrame(subject_row).T]*len(df_data_aug), ignore_index=True)
            
            df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)
            #df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)
            
            # Append the augmented dataframe to the original dataframe
            tr = pd.concat([tr, df_data_aug], ignore_index=True)
            
            print('After spk ', str(spk), ' tr shape is : ', tr.shape)
#             print(tr.isnull().sum())
#             raise KeyboardInterrupt
#         tr.to_csv("mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value), index=False)
    else:
        # If we are using lambda data augmentation, these columns were already removed 
        tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    train_y = tr[obj].astype(pd.np.float32) # training labels 
    train_weight = tr['spcount'] ** -0.5 # training weight
    tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)
    print('train_weight : ', train_weight)

    ##te_w = all_features_labels[idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##te = pd.merge(all_features_labels[idx], te_w, on='subject_id')
    
    # Drop the measurements that are used in the training of this fold, so we keep [idx] instead of [~idx]
    te = all_features_labels[idx].drop(['fold_id'], axis=1)
    print('te !!! : ', te)
    test_weight = te['spcount'] ** -0.5 # test weight 
    test_y = te[obj].astype(pd.np.float32) # testing labels
    #sub = te['subject_id']
    sid = te.subject_id
    test_measurement_id = te.measurement_id
    te = te.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    # XGBoost 
    clf = xgb.XGBRegressor(**best_params)
    #clf = xgb.XGBClassifier(**params)

    # Random Forest Regressor 
    #clf = RandomForestRegressor(**best_params)
    #clf.fit(tr, train_y, sample_weight=train_weight)

    # Fit for the xgboost 
    clf.fit(
        tr, train_y,
        sample_weight=train_weight,
        eval_set=[(tr, train_y), (te, test_y)],
        #eval_metric=',
        sample_weight_eval_set=[train_weight, test_weight],
        verbose=0,
        early_stopping_rounds=100
    )
    pred = clf.predict(te).clip(0, 4)
    print('pred : ', pred)
            

After spk  1007  tr shape is :  (29111, 1679)
After spk  1019  tr shape is :  (29706, 1679)
After spk  1020  tr shape is :  (41487, 1679)
After spk  1048  tr shape is :  (43972, 1679)
After spk  1049  tr shape is :  (45988, 1679)
After spk  1038  tr shape is :  (59354, 1679)
After spk  1032  tr shape is :  (68945, 1679)
After spk  1006  tr shape is :  (69323, 1679)
After spk  1023  tr shape is :  (72644, 1679)
After spk  1034  tr shape is :  (73079, 1679)
After spk  1046  tr shape is :  (74457, 1679)
After spk  1043  tr shape is :  (74757, 1679)
After spk  1004  tr shape is :  (76710, 1679)


/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


train_weight :  0        0.130189
1        0.130189
2        0.130189
3        0.130189
4        0.130189
           ...   
76705    0.258199
76706    0.258199
76707    0.258199
76708    0.258199
76709    0.258199
Name: spcount, Length: 76710, dtype: object
te !!! :        X2_abs_energy  X2_abs_sum_of_changes  X2_agg_autocorr_mean  \
115     -192.094624            -368.347709             -0.004121   
116     -310.502489            -864.995659              0.085568   
117      -35.704579             108.350045             -0.006157   
118     -280.078865            -686.556938              0.048710   
119     -125.802281             207.746773             -0.059889   
...             ...                    ...                   ...   
1411    -680.675063            -512.064364              0.101239   
1412    -379.461455             639.783781             -0.055631   
1413     -97.353514             854.160734             -0.034258   
1414    -618.977736            -313.060702          

/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/home/mpgill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


pred :  [0.         0.04718658 0.         0.         0.         0.
 0.         0.12364379 0.         0.09805939 0.18257746 0.02346808
 0.11244491 0.05892476 0.         0.         0.1250911  0.32890075
 0.         0.21537888 0.10175487 0.17699564 0.         0.13635418
 0.         0.18448114 0.02076581 0.05165336 0.07526433 0.13712996
 0.13893434 0.         0.         0.00348854 0.         0.11611366
 0.10865381 0.         0.12714317 0.08164918 0.         0.08173203
 0.19980666 0.1086795  0.07183036 0.02074012 0.         0.14965221
 0.22722271 0.08153096 0.31454092 0.         0.09612843 0.
 0.00530693 0.         0.14987281 0.03177106 0.         0.
 0.         0.02974322 0.         0.3247292  0.         0.
 0.02545002 0.2129114  0.         0.10002038 0.         0.0872775
 0.         0.07398161 0.         0.19439524 0.         0.
 0.10411763 0.         0.         0.16227165 0.05165336 0.
 0.18792757 0.08153096 0.11611366 0.         0.         0.
 0.16227165 0.02130097 0.1964755  0.        

KeyboardInterrupt: 

In [383]:
subject_row.append(pd.Series(mean_spcount, index=["spcount"]))

spk__1004          1
spk__1006          0
spk__1007          0
spk__1019          0
spk__1020          0
spk__1023          0
spk__1032          0
spk__1034          0
spk__1038          0
spk__1043          0
spk__1046          0
spk__1048          0
spk__1049          0
spcount      59.3207
dtype: object

In [394]:
train_y.isnull().values.any()
# tr.isnull().sum().to_string()
# tr.isnull().sum().sum()
print(te.isnull().T.any().T.sum())

0


In [360]:
tr['spcount']

KeyError: 'spcount'

In [358]:
print(train_weight.isnull().sum())

75565


In [341]:
print(clf)
for item in zip(feature_list_transform, xgb.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=0.5,
       colsample_bynode=1, colsample_bytree=0.8, gamma=1.0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.3, max_delta_step=0, max_depth=2,
       min_child_weight=0.5, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=100.0, scale_pos_weight=1, silent=False, subsample=1.0,
       tree_method=None, validate_parameters=False, verbosity=None)


NameError: name 'feature_list_transform' is not defined

In [336]:
booster = model.get_booster()
dump = booster.get_dump()[0]
assert not ('\nnan\n' in dump or '\n-nan\n' in dump or '\ninf\n' in dump or '\n-inf\n' in dump), "Got non-finite weights from fit: %s" % str(dump)

NameError: name 'model' is not defined

In [317]:
tr_subject.iloc[0,tr_subject.columns.str.startswith("spk_")]

spk__1004    0
spk__1006    0
spk__1007    0
spk__1019    1
spk__1020    0
spk__1023    0
spk__1032    0
spk__1034    0
spk__1038    0
spk__1043    0
spk__1046    0
spk__1048    0
spk__1049    0
Name: 237, dtype: object

In [81]:
# def data_aug_lambda(measurement1, measurement2, lambda_value):
#     modDfObj1 = measurement1.apply(lambda x: x * lambda_value, axis=1)
#     modDfObj2 = measurement2.apply(lambda x: x * (1-lambda_value), axis=1)
#     mea_new = modDfObj1.add(modDfObj2)
#     return mea_new

In [175]:
def data_aug_lambda(modDfObj1, modDfObj2):
    df_data_aug = []

    # First loop to go over the rows
    for index, measurement1 in modDfObj1.iterrows():
        # Second rows to go over the loop except the same two rows
        for index2, measurement2 in modDfObj2.iterrows():
            if index >= index2:
                continue
            df_data_aug.append(measurement1.add(measurement2).to_list())
    df_data_aug = pd.DataFrame(df_data_aug, columns=modDfObj1.columns)

    
    return df_data_aug

In [272]:
columns_to_exclude

['measurement_id',
 'subject_id',
 'spcount',
 'spk__1004',
 'spk__1006',
 'spk__1007',
 'spk__1019',
 'spk__1020',
 'spk__1023',
 'spk__1032',
 'spk__1034',
 'spk__1038',
 'spk__1043',
 'spk__1046',
 'spk__1048',
 'spk__1049']

## Old drafts

In [412]:
salut1 = pd.DataFrame([['a', 6, 4, 10, 2], ['a', 4, 10, 12, 4]], columns=['subject_id','X','Y','Z','label'])
# salut2 = pd.DataFrame([[6, 4, 10, 2], [4, 10, 12, 4], [10, 8, 16, 22], [2, 10, 16, 20]])
lambda_value = 0.5

modDfObj1 = salut1[salut1.columns.difference(['subject_id'])].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
modDfObj2 = salut1[salut1.columns.difference(['subject_id'])].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
display(salut1)
display(salut2)
display('modDfObj1 : ', modDfObj1)
display('modDfObj2 : ', modDfObj2)
df_data_aug2 = data_aug_lambda(modDfObj1, modDfObj2)
display(df_data_aug2)
# df_data_aug2 = pd.concat([df_data_aug2, salut1['subject']], axis=1)
subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]

# Select the first row of spk_ elements 
subjects = salut1.loc[0,salut1.columns.str.startswith("subject_id")]

# Duplicate that first row for the number of elements we need
full_subjects = pd.concat([subjects]*len(df_data_aug2), ignore_index=True)
full_subjects = pd.DataFrame(full_subjects)
full_subjects.columns = ['subject_id']
# Assign the column name 
display(full_subjects)
print(df_data_aug2.keys())
# Merge the data augmented with the spk information
df_data_aug2 = pd.concat([df_data_aug2, full_subjects], axis=1)
print(df_data_aug2.keys())
print(df_data_aug2)
result = pd.concat([salut1, df_data_aug2], ignore_index=True)
display(result)

,subject_id,X,Y,Z,label
0,a,6,4,10,2
1,a,4,10,12,4


,0,1,2,3
0,6,4,10,2
1,4,10,12,4
2,10,8,16,22
3,2,10,16,20


'modDfObj1 : '

,X,Y,Z,label
0,3,2,5,1
1,2,5,6,2


'modDfObj2 : '

...

data aug shape :  (1, 4)


,X,Y,Z,label
0,5,7,11,3


,subject_id
0,a


Index(['X', 'Y', 'Z', 'label'], dtype='object')
Index(['X', 'Y', 'Z', 'label', 'subject_id'], dtype='object')
   X  Y   Z  label subject_id
0  5  7  11      3          a


,subject_id,X,Y,Z,label
0,a,6,4,10,2
1,a,4,10,12,4
2,a,5,7,11,3


In [401]:
salut1 = pd.DataFrame([['a', 6, 4, 10, 2], ['b', 4, 10, 12, 4], ['c', 10, 8, 16, 22], ['d',2, 10, 16, 20]], columns=['subject','1','2','3','4'])
# salut2 = pd.DataFrame([[6, 4, 10, 2], [4, 10, 12, 4], [10, 8, 16, 22], [2, 10, 16, 20]])
lambda_value = 0.5

modDfObj1 = salut1[salut1.columns.difference(['subject'])].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
modDfObj2 = salut1[salut1.columns.difference(['subject'])].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
display(salut1)
display(salut2)
display('modDfObj1 : ', modDfObj1)
display('modDfObj2 : ', modDfObj2)
df_data_aug2 = data_aug_lambda(modDfObj1, modDfObj2)
display(df_data_aug2)
# df_data_aug2 = pd.concat([df_data_aug2, salut1['subject']], axis=1)
subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]

# Select the first row of spk_ elements 
subjects = salut1.loc[0,salut1.columns.str.startswith("subject")]

# Duplicate that first row for the number of elements we need
full_subjects = pd.concat([subjects]*len(df_data_aug2), ignore_index=True)
full_subjects = pd.DataFrame(full_subjects)
# full_subjects.columns = subjects_columns_title
# Assign the column name 
display(full_subjects)
print(df_data_aug2.keys())
# Merge the data augmented with the spk information
df_data_aug2 = pd.concat([df_data_aug2, full_subjects], axis=1)
print(df_data_aug2.keys())
print(df_data_aug2)
result = pd.concat([modDfObj1, df_data_aug2], ignore_index=True)
display(result)

,subject,1,2,3,4
0,a,6,4,10,2
1,b,4,10,12,4
2,c,10,8,16,22
3,d,2,10,16,20


,0,1,2,3
0,6,4,10,2
1,4,10,12,4
2,10,8,16,22
3,2,10,16,20


'modDfObj1 : '

,1,2,3,4
0,3,2,5,1
1,2,5,6,2
2,5,4,8,11
3,1,5,8,10


'modDfObj2 : '

...

data aug shape :  (6, 4)


,1,2,3,4
0,5,7,11,3
1,8,6,13,12
2,4,7,13,11
3,7,9,14,13
4,3,10,14,12
5,6,9,16,21


,0
0,a
1,a
2,a
3,a
4,a
5,a


Index(['1', '2', '3', '4'], dtype='object')
Index(['1', '2', '3', '4', 0], dtype='object')
   1   2   3   4  0
0  5   7  11   3  a
1  8   6  13  12  a
2  4   7  13  11  a
3  7   9  14  13  a
4  3  10  14  12  a
5  6   9  16  21  a


,1,2,3,4,0
0,3,2,5,1,NaN
1,2,5,6,2,NaN
2,5,4,8,11,NaN
3,1,5,8,10,NaN
4,5,7,11,3,a
5,8,6,13,12,a
6,4,7,13,11,a
7,7,9,14,13,a
8,3,10,14,12,a
9,6,9,16,21,a


In [114]:
lambda_value = 0.3 

for i in range(5):
    ##test = pd.read_csv(sys.argv[i]).squeeze()
    ##idx = all_features_labels['measurement_id'].isin(test)

    # Filter all_features_labels to only the measurements of the current fold
    idx = all_features_labels['fold_id'] == i

    ##tr_w = all_features_labels[~idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##tr = pd.merge(all_features_labels[~idx], tr_w, on='subject_id')
    
    # Drop the measurements that are not from the current fold 
    tr = all_features_labels[~idx].drop(['fold_id'], axis=1)
    train_weight = tr['spcount'] ** -0.5 # training weight 
    print(tr)
    # Data augmentation with a lambda 
    if lambda_value is not None: 
        df2 = df[tr.columns.difference(['subject_id', 'spcount'])]
        df2 = df2.mul(lambda_value)
        tr.append(tr.mul(lambda_value))
        
    train_y = tr[obj].astype(pd.np.float32) # training labels 
    tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    ##te_w = all_features_labels[idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##te = pd.merge(all_features_labels[idx], te_w, on='subject_id')
    
    # Drop the measurements that are used in the training of this fold, so we keep [idx] instead of [~idx]
    te = all_features_labels[idx].drop(['fold_id'], axis=1)
    test_weight = te['spcount'] ** -0.5 # test weight 
    test_y = te[obj].astype(pd.np.float32) # testing labels
    #sub = te['subject_id']
    sid = te.subject_id
    test_measurement_id = te.measurement_id
    te = te.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

      X2_abs_energy  X2_abs_sum_of_changes  X2_agg_autocorr_mean  \
0       -291.834789            -724.139702              0.071326   
1        318.026104            1054.088082             -0.039085   
2       -203.416378            -163.318657             -0.013062   
3       -262.598660            -458.680961             -0.018761   
4       -182.269901             -91.254692             -0.006309   
...             ...                    ...                   ...   
1457    -831.467137           -1360.095126              0.217847   
1458    -708.415480            -635.210116             -0.016788   
1459    -301.895384             127.097013             -0.037851   
1460    -618.234831            -822.404696              0.054343   
1461    -775.501738           -1083.683993              0.082838   

      X2_agg_autocorr_median  X2_agg_autocorr_var  \
0                   0.085979            -0.000472   
1                  -0.053770             0.000831   
2                  -0.00

TypeError: can't multiply sequence by non-int of type 'float'

In [183]:
numerics = ['int16', 'int32', 'int64']

newdf = tr.select_dtypes(include=numerics)
newdf

,subject_id,spcount
0,1007,59
1,1007,59
2,1007,59
3,1007,59
4,1007,59
...,...,...
1457,1004,16
1458,1004,16
1459,1004,16
1460,1004,16


In [86]:
test_y

59      0.672241
60      0.672241
61      1.672241
62     -0.327759
63     -0.327759
          ...   
1441   -1.000000
1442   -1.000000
1443   -1.000000
1444   -1.000000
1445   -1.000000
Name: tremor, Length: 266, dtype: float32

In [72]:
sid

59      1007
60      1007
61      1007
62      1007
63      1007
        ... 
1441    1004
1442    1004
1443    1004
1444    1004
1445    1004
Name: subject_id, Length: 266, dtype: int64

In [73]:
tid

59      40a4dedc-1de0-4c26-bad5-f680f72de30f
60      45419562-4d6f-44fe-ab6c-be92d90b1d4a
61      4679db13-6ca0-4a40-971b-06d537cb65cf
62      4731d55d-9e68-472b-a415-a7a9de6776fa
63      5131b2bd-e44c-4190-90aa-f3a9b9efa6fa
                        ...                 
1441    184ffe2e-f6c6-414d-ae43-d01bcd5568d9
1442    1d624788-41fd-4c7d-b2c3-2fed920457f6
1443    1e051b85-eca9-4831-b1e3-444a4cb18269
1444    32184eb0-2e4c-49a2-8d63-c1a67f9e17a3
1445    49202934-d5f5-4e83-8c61-c04ed0ae4a35
Name: measurement_id, Length: 266, dtype: object

In [76]:
tr.keys().to_list()

['X2_abs_energy',
 'X2_abs_sum_of_changes',
 'X2_agg_autocorr_mean',
 'X2_agg_autocorr_median',
 'X2_agg_autocorr_var',
 'X2_augmented_dickey_fuller_pvalue',
 'X2_augmented_dickey_fuller_teststat',
 'X2_augmented_dickey_fuller_usedlag',
 'X2_autocorrelation_1',
 'X2_autocorrelation_10',
 'X2_autocorrelation_100',
 'X2_autocorrelation_1000',
 'X2_autocorrelation_2',
 'X2_autocorrelation_3',
 'X2_autocorrelation_4',
 'X2_autocorrelation_5',
 'X2_autocorrelation_50',
 'X2_autocorrelation_6',
 'X2_autocorrelation_7',
 'X2_autocorrelation_8',
 'X2_autocorrelation_9',
 'X2_binned_entropy_10',
 'X2_binned_entropy_100',
 'X2_binned_entropy_20',
 'X2_binned_entropy_5',
 'X2_binned_entropy_50',
 'X2_binned_entropy_80',
 'X2_c3_1',
 'X2_c3_10',
 'X2_c3_100',
 'X2_c3_2',
 'X2_c3_3',
 'X2_c3_4',
 'X2_c3_5',
 'X2_cid_ce_0',
 'X2_cid_ce_1',
 'X2_count_above_mean',
 'X2_count_below_mean',
 'X2_fft_10_ang',
 'X2_fft_10_imag',
 'X2_fft_10_real',
 'X2_fft_11_ang',
 'X2_fft_11_imag',
 'X2_fft_11_real',
 '

In [77]:
tr.X2_abs_energy

0      -291.834778
1       318.026093
2      -203.416382
3      -262.598663
4      -182.269897
           ...    
1457   -831.467163
1458   -708.415466
1459   -301.895386
1460   -618.234802
1461   -775.501709
Name: X2_abs_energy, Length: 1196, dtype: float32

In [78]:
tr.X_abs_energy

0       -4221.129883
1       19108.812500
2        1490.269531
3       -6997.208984
4         985.650391
            ...     
1457    13229.256836
1458   -15638.213867
1459      604.454346
1460    -5448.446777
1461     9931.563477
Name: X_abs_energy, Length: 1196, dtype: float32

In [80]:
tr_y

0      -0.327759
1       0.672241
2       0.672241
3      -0.327759
4       0.672241
          ...   
1457   -1.000000
1458   -1.000000
1459   -1.000000
1460   -1.000000
1461    2.000000
Name: tremor, Length: 1196, dtype: float32

In [173]:
data = {'subject_id':[1,2,3,4], 'spcount':[6,7,8,10], 'feat1':[5.6,2.4,6.7,4.5], 'label':[1.0,5.0,6.0,9.0]} 

# Create DataFrame 
df = pd.DataFrame(data, columns=['subject_id', 'spcount', 'feat1', 'label'])

In [174]:
df

,subject_id,spcount,feat1,label
0,1,6,5.6,1.0
1,2,7,2.4,5.0
2,3,8,6.7,6.0
3,4,10,4.5,9.0


In [175]:
numerics = ['int16', 'int32', 'int64']

newdf = df.select_dtypes(include=numerics)
newdf

,subject_id,spcount
0,1,6
1,2,7
2,3,8
3,4,10


In [188]:
df2 = df[df.columns.difference(['subject_id', 'spcount'])]

print(df[['subject_id','spcount']])
df2 = df2.mul(lambda_value)
print(df2)

result = pd.concat([df[['subject_id','spcount']], df2], axis=1, sort=False)
display(result)
# df3 = pd.merge(df[['subject_id','spcount']], df2)
# df3
#df.append(df.mul(0.5), ignore_index=True)

   subject_id  spcount
0           1        6
1           2        7
2           3        8
3           4       10
   feat1  label
0   1.68    0.3
1   0.72    1.5
2   2.01    1.8
3   1.35    2.7


,subject_id,spcount,feat1,label
0,1,6,1.68,0.3
1,2,7,0.72,1.5
2,3,8,2.01,1.8
3,4,10,1.35,2.7


In [189]:
df

,subject_id,spcount,feat1,label
0,1,6,5.6,1.0
1,2,7,2.4,5.0
2,3,8,6.7,6.0
3,4,10,4.5,9.0


In [198]:
allo = pd.concat([df, result], ignore_index=True)

In [199]:
allo

,subject_id,spcount,feat1,label
0,1,6,5.60,1.0
1,2,7,2.40,5.0
2,3,8,6.70,6.0
3,4,10,4.50,9.0
4,1,6,1.68,0.3
5,2,7,0.72,1.5
6,3,8,2.01,1.8
7,4,10,1.35,2.7
